<a href="https://colab.research.google.com/github/gorky93/stackoverflow_analysis/blob/main/stackoverflow24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git config --global user.email "sdmathdeep@gmail.com"
!git config --global user.name "gorky93"

In [3]:
!git clone https://github.com/gorky93/stackoverflow_analysis.git

Cloning into 'stackoverflow_analysis'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


In [20]:
import zipfile
zip_path = '/content/drive/My Drive/Project Data/stack-overflow-developer-survey-2024.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data')

In [19]:
!ls '/content/drive/My Drive/Project Data/'

'CBC data_for_meandeley_csv1.csv'   stack-overflow-developer-survey-2024.zip


In [22]:
!ls

data  drive  sample_data  stackoverflow_analysis


In [23]:
import os
print(os.listdir('./data'))

['survey_results_public.csv', '2024 Developer Survey.pdf', 'survey_results_schema.csv']
